In [25]:
from math import sqrt, log, exp
from scipy.stats import norm
from scipy.optimize import newton
from datetime import datetime

# import the option_price method from the Black_Scholes_Option_Pricing_Model notebook
%run './Back-End/Pricing Models/Black-Scholes/Black_Scholes_Option_Pricing_Model.ipynb'


Call Price: $ 3.94
Put Price: $ 1.16


In [26]:
def calculate_implied_volatility(S, K, T, market_price, option_type, date_str=None):
    """
    Calculate the implied volatility using Black-Scholes formula and Newton-Raphson method
    when market sources for volatility are unavailable.
    """
    # Define the objective function and its derivative
    def objective_function(volatility):
        """
        Objective function: Difference between the calculated and market prices.
        """
        price = option_price(S, K, T, volatility, option_type, date_str)
        print(f"Objective Function - Volatility: {volatility}, Price: {price}, Market Price: {market_price}")
        return price - market_price

    def vega(volatility):
        """
        Vega (the derivative of the option price with respect to volatility).
        """
        # Calculate the rate using option_price logic for consistency
        r = fetch_risk_free_rate(date_str)
        d1 = (log(S / K) + (r + 0.5 * volatility ** 2) * T) / (volatility * sqrt(T))
        return S * sqrt(T) * norm.pdf(d1)

    # Fetch the risk-free rate
    def fetch_risk_free_rate(date_str):
        """
        Fetch the risk-free rate using the same logic as the option_price function.
        """
        treasury_rate = yf.Ticker("^TNX").history(start='1900-1-1')
        treasury_rate.index = pd.to_datetime(treasury_rate.index).tz_localize(None)
        if date_str is None:
            return treasury_rate['Close'].iloc[-1] / 100
        else:
            date = datetime.strptime(date_str, "%Y-%m-%d")
            nearest_index = treasury_rate.index.get_indexer([pd.Timestamp(date)], method='nearest')[0]
            return treasury_rate.iloc[nearest_index]['Close'] / 100

    # Solve for implied volatility
    volatility_guess = 0.2
    try:
        implied_volatility = newton(objective_function, volatility_guess, fprime=vega)
        return implied_volatility
    except RuntimeError as e:
        print(f"Newton method failed to converge. Error: {e}")
        return "Failed to converge: Try another initial guess"


In [27]:
# Test
S = 100 # Underlying asset price
K = 100 # Strike price
T = 1 # One year until expiry
market_price = 12 # Retrieve this from your data
option_type = 'C' # This can either be 'C' or 'P'
date_str = '2024-01-01' # Example date, can be removed for the last available rate

implied_volatility = calculate_implied_volatility(S, K, T, market_price, option_type, date_str)
print("Implied Volatility: ", implied_volatility)


Objective Function - Volatility: 0.2, Price: 9.897066333083465, Market Price: 12
Objective Function - Volatility: 0.25509454044537544, Price: 12.005686960678617, Market Price: 12
Objective Function - Volatility: 0.2549461973379464, Price: 12.000000012694379, Market Price: 12
Implied Volatility:  0.25494619700681487
